# Load and save ERA5-Land Daily Aggregated data over an Area of Interest


## Requirements: 

- __Google Earth Engine account__. Sign up [here](https://earthengine.google.com/signup/).
- __Reference DEM__ (.tif, .nc, or similar file readable by xarray). 

In [ ]:
import ee
import geedim as gd
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
from tqdm.auto import tqdm
import xarray as xr
import rioxarray as rxr
import geojson
from shapely.geometry import Polygon
import datetime

## Authenticate and initialize Google Earth Engine

In [ ]:
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

## Define paths in directory and snow depth date

In [ ]:
# Site info used for output file names
site_name = 'MCS'
date = '2024-03-15'

out_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/SnowMaL/study-sites/MCS/'
refdem_fn = '/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/study-sites/MCS/refdem/MCS_REFDEM_WGS84.tif'

In [ ]:
# Create AOI based on bounding box of reference DEM
refdem = rxr.open_rasterio(refdem_fn).squeeze()
xmin, xmax, ymin, ymax = [np.min(refdem.x.data), np.max(refdem.x.data), np.min(refdem.y.data), np.max(refdem.y.data)]
aoi = Polygon([[xmin, ymin], [xmax, ymin], [xmax, ymax], [xmin, ymax], [xmin, ymin]])
aoi_buffer = aoi.buffer(1e3)
aoi_buffer_gdf = gpd.GeoDataFrame(geometry=[aoi_buffer], crs=f"EPSG:{refdem.rio.crs.to_epsg()}")
aoi_buffer

## Query GEE for ERA5-Land

In [ ]:
# Adjust AOI for GEE querying
aoi_buffer_wgs_gdf = aoi_buffer_gdf.to_crs('EPSG:4326')
aoi_ee = ee.Geometry.Polygon(list(zip(aoi_buffer_wgs_gdf.geometry[0].exterior.coords.xy[0], 
                                      aoi_buffer_wgs_gdf.geometry[0].exterior.coords.xy[1])))

# Define date range
dt = datetime.datetime(int(date[0:4]), int(date[5:7]), int(date[8:]))
if dt.month < 10:
    water_year = dt.year
else:
    water_year = dt.year + 1
start_date = f"{water_year-1}-10-01"
end_date = date

# Define bands to extract from ERA5-Land
# See all data bands in the GEE documentation here: 
# https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_DAILY_AGGR#bands
bands = ['temperature_2m', 
         'total_precipitation_sum', 
         'snowfall_sum', 
         'snowmelt_sum'] 

In [ ]:
# Define output file name
out_fn = os.path.join(out_dir, f"{site_name}_{date.replace('-','')}_ERA5-Land.tif")

# Check if output file already exists
if os.path.exists(out_fn):
    print('File already exists in file, skipping...')

else:

    # Grab ERA5-Land daily image collection
    era5_land = (ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")
                             .filter(ee.Filter.date(start_date, end_date))
                             .filterBounds(aoi_ee))
    
    # Clip to AOI
    def clip_to_aoi(im):
        return im.clip(aoi_ee)
    era5_land = era5_land.map(clip_to_aoi)
    
    # Calculate cumulative snowfall, snowmelt, and precipitation
    snowfall_cumsum = era5_land.select('snowfall_sum').reduce(ee.Reducer.sum()).rename('snowfall_cumsum')
    snowmelt_cumsum = era5_land.select('snowmelt_sum').reduce(ee.Reducer.sum()).rename('snowmelt_cumsum')
    precip_cumsum = era5_land.select('total_precipitation_sum').reduce(ee.Reducer.sum()).rename('precip_cumsum')

    # Calculate PDDs
    def calculate_pdd(image):
        temp_2m = image.select('temperature_2m')
        pdd = temp_2m.subtract(273.15).max(0)  # Convert from Kelvin to Celsius and calculate PDD
        return image.addBands(pdd.rename('PDD'))
    era5_land = era5_land.map(calculate_pdd)
    
    # Calculate the cumulative sum of PDDs
    pdd_cumsum = era5_land.select('PDD').reduce(ee.Reducer.sum()).rename('PDD_cumsum')
    
    # Combine into one image
    final_image = snowfall_cumsum.addBands([snowmelt_cumsum, precip_cumsum, pdd_cumsum])
    
    # Download to file
    final_image_gd = gd.MaskedImage(final_image)
    final_image_gd.download(out_fn, region=aoi_ee, scale=100, crs=f"EPSG:{refdem.rio.crs.to_epsg()}")
    print('ERA5-Land image saved to file:', out_fn)

# Open image, adjust bands, and plot
era_xr = xr.open_dataset(out_fn)
era_ds = xr.Dataset(data_vars=dict(snowfall_cumsum=(["y", "x"], era_xr.band_data.data[0]),
                                   snowmelt_cumsum=(["y", "x"], era_xr.band_data.data[1]),
                                   precip_cumsum=(["y", "x"], era_xr.band_data.data[2]),
                                   pdd_cumsum=(["y", "x"], era_xr.band_data.data[3])
                                  ),
                    coords=dict(x=era_xr.x,
                                y=era_xr.y
                               )
                   )
data_vars = list(era_ds.data_vars)
fig, ax = plt.subplots(len(data_vars), 1, figsize=(6, 6*len(data_vars)), sharex=True, sharey=True)
cmaps = ['Blues', 'Reds', 'Greens', 'YlOrRd']
for i, var in enumerate(data_vars):
    im = ax[i].imshow(era_ds[var].data, cmap=cmaps[i],
                      extent=(np.min(era_ds.x.data), np.max(era_ds.x.data), 
                              np.min(era_ds.y.data), np.max(era_ds.y.data)))
    ax[i].set_title(var)
    fig.colorbar(im, ax=ax[i])

plt.show()